<a href="https://colab.research.google.com/github/jack978397/Colab-AI-Transcribe-Translate/blob/main/1_Whisper_Transcription_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. 修正安裝指令 (加入 Google AI 庫) ---
!pip install openai-whisper yt-dlp srt "moviepy<2.0.0" Gradio google-generativeai
!apt-get install -y ffmpeg

print("\n--- API 環境安裝完成！ ---")


In [ ]:
import whisper
import torch
import os
import re
import datetime
import srt
import yt_dlp

# --- 2. 字幕處理工具 ---
def format_srt_time(seconds):
    if seconds < 0: seconds = 0
    delta = datetime.timedelta(seconds=seconds)
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = delta.microseconds // 1000
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

def refine_srt_segments(segments, max_chars=18):
    if not segments: return []
    pre_filtered = []
    last_text = None
    for seg in segments:
        text = seg['text'].strip()
        if not text: continue
        if text == last_text and (seg['end'] - seg['start']) < 0.8:
            if pre_filtered: pre_filtered[-1]['end'] = seg['end']
            continue
        pre_filtered.append(seg)
        last_text = text

    final_segments = []
    punctuation_pattern = r'([，。、？！,.\!\?])'
    for segment in pre_filtered:
        text = segment['text'].strip()
        if len(text) <= max_chars:
            final_segments.append(segment)
            continue
        start_time, end_time = segment['start'], segment['end']
        duration = end_time - start_time
        parts = re.split(punctuation_pattern, text)
        sub_sentences = [p for p in parts if p.strip()]

        current_sentence = ""
        total_len = sum(len(s) for s in sub_sentences)
        current_start = start_time
        for sub in sub_sentences:
            if len(current_sentence) > 0 and len(current_sentence + sub) > max_chars:
                ratio = len(current_sentence) / total_len
                new_end = current_start + (ratio * duration)
                final_segments.append({'start': current_start, 'end': new_end, 'text': current_sentence})
                current_start = new_end
                current_sentence = sub
            else:
                current_sentence += (" " if current_sentence and not re.match(r'^[，。、？！]', sub) else "") + sub
        if current_sentence:
            final_segments.append({'start': current_start, 'end': end_time, 'text': current_sentence})

    deduped = []
    for s in final_segments:
        if not deduped or s['text'] != deduped[-1]['text']:
            deduped.append(s)
        else:
            deduped[-1]['end'] = s['end']
    for i, seg in enumerate(deduped): seg['id'] = i
    return deduped

def generate_srt_from_segments(segments):
    srt_content = []
    for i, segment in enumerate(segments):
        start_time, end_time = format_srt_time(segment['start']), format_srt_time(segment['end'])
        text = segment['text'].strip()
        if text:
            srt_content.append(str(i + 1))
            srt_content.append(f"{start_time} --> {end_time}")
            srt_content.append(text)
            srt_content.append("")
    return "\n".join(srt_content)

def fix_and_process_srt(srt_content, remove_punctuation=True):
    try:
        cleaned_content = "\n".join(line for line in srt_content.strip().splitlines() if line.strip())
        subs = list(srt.parse(cleaned_content))
        punctuations_to_remove = '.,。?!！?？'
        for sub in subs:
            mod = sub.content.strip()
            if remove_punctuation: mod = mod.rstrip(punctuations_to_remove)
            sub.content = mod.strip()
        return srt.compose(subs)
    except: return srt_content

# --- 4. 轉錄辨識引擎 ---
class WhisperEngineCloud:
    def __init__(self):
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_size = None

    def load_model(self, model_size="large"):
        if self.model is not None and self.model_size == model_size: return
        print(f"正在載入模型 {model_size}...")
        self.model = whisper.load_model(model_size, device=self.device)
        self.model_size = model_size
        print("載入成功。")

    def transcribe(self, audio_path, prompt=""):
        return self.model.transcribe(
            audio_path,
            initial_prompt=prompt,
            verbose=True,
            compression_ratio_threshold=2.4,
            no_speech_threshold=0.6
        )

engine = WhisperEngineCloud()
print("\n--- 轉錄引擎 (精簡工作站版) 載入完畢！ ---")


In [ ]:
import gradio as gr
import gc
import torch
import os

def process_transcription(file_input, yt_input, input_type, model_size, prompt, remove_punc, progress=gr.Progress()):
    gc.collect()
    torch.cuda.empty_cache()
    progress(0, desc="🚀 正在準備引擎...")

    if input_type == "檔案上傳":
        if not file_input: return "❌ 錯誤：請先上傳檔案。", None
        audio_source, output_video = file_input, None
    else:
        if not yt_input: return "❌ 錯誤：請輸入 YouTube 網址", None
        progress(0.1, desc="正在從 YouTube 雲端抓取...")
        ydl_opts = {'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best', 'outtmpl': 'downloaded_video.%(ext)s', 'noplaylist': True}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl: ydl.download([yt_input])
        audio_source, output_video = 'downloaded_video.mp4', 'downloaded_video.mp4'

    engine.load_model(model_size)
    progress(0.4, desc=f"🔥 正在辨識 ({model_size} 模型)...")
    result = engine.transcribe(audio_source, prompt)

    progress(0.8, desc="格式佈置中...")
    refined = refine_srt_segments(result['segments'])
    srt_content = generate_srt_from_segments(refined)
    final_srt = fix_and_process_srt(srt_content, remove_punctuation=remove_punc)

    srt_output_path = os.path.abspath("original_transcript.srt")
    with open(srt_output_path, "w", encoding="utf-8") as f: f.write(final_srt)

    out_files = [srt_output_path]
    if output_video: out_files.append(os.path.abspath(output_video))
    return final_srt, out_files

# --- 單純轉錄 UI ---
with gr.Blocks(title="Whisper Pro 轉錄工作站") as demo:
    gr.Markdown("# 🚀 Whisper Pro 影音轉錄與校稿工作站")

    with gr.Row():
        with gr.Column(scale=1):
            t_mode = gr.Radio(["檔案上傳", "YouTube 連結"], label="輸入模式", value="檔案上傳")
            t_file = gr.File(label="上傳")
            t_url = gr.Textbox(label="網址", visible=False)
            t_model = gr.Dropdown(["base", "medium", "large"], label="使用模型 (建議用 large)", value="large")
            t_btn = gr.Button("🔥 開始轉錄辨識", variant="primary")

        with gr.Column(scale=2):
            gr.Markdown("### ✍️ 原音字幕預覽 (校稿區)")
            with gr.Row():
                f_in = gr.Textbox(placeholder="尋找", label="尋找", scale=2)
                r_in = gr.Textbox(placeholder="取代", label="取代", scale=2)
                rep_btn = gr.Button("全部取代", scale=1)
            t_preview = gr.Textbox(label="字幕內容（可在這裡直接手動修改）", lines=18, interactive=True)
            t_out = gr.File(label="📥 下載 SRT 字幕 (右鍵另存可存到指定夾)", file_count="multiple")

    def t_toggle(choice):
        if choice == "YouTube 連結": return gr.update(visible=False), gr.update(visible=True)
        return gr.update(visible=True), gr.update(visible=False)
    t_mode.change(t_toggle, t_mode, [t_file, t_url])

    def do_replace(text, f, r):
        new_text = text.replace(f, r) if f else text
        path = os.path.abspath("original_transcript.srt")
        with open(path, "w", encoding="utf-8") as file: file.write(new_text)
        return new_text, [path]

    rep_btn.click(do_replace, [t_preview, f_in, r_in], [t_preview, t_out])
    t_btn.click(process_transcription, [t_file, t_url, t_mode, t_model, gr.State(""), gr.State(True)], [t_preview, t_out])

demo.queue().launch(debug=True, share=True)
